Pseudocode for Permutation Core Butson-Type Complex Hadamard Matrices:

* Select dimension $n$ for matrix
* Generate multiples of $\Phi_n(x)$ with sum of coefficient magnitudes equal to $n$
* Normalize so that they're all positive coefficients by multiplying by $x^{n/2}$ for the negative ones